In [1]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from transformers.Rebalancer import Rebalancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from utils.DataHelper import DataHelper
from utils.Common import Config

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [3]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label)
new_df = pc.getFrame()

In [4]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [5]:
# pass feature to pipeline and convert it to numerical data
X = DataPipeline(Config.num_attribs,Config.cat_attribs).process(X)

In [6]:
X.isna().sum().sum()

0

In [7]:
Y.value_counts().tolist()

[14246, 2201]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=Config.test_size, stratify=Y)


In [9]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight ='balanced', classes =y_train.unique(),  y = y_train)
class_weights_dict = dict(zip(y_train.unique(),class_weights))


In [10]:
# Create param grid
param_grid = [
    {
    'penalty' : ['l2'],
    'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver' : ['lbfgs', 'newton-cg','sag', 'saga' ]
    }
]


clf = GridSearchCV(estimator=LogisticRegression(class_weight ='balanced' ),param_grid=param_grid,n_jobs=10,cv=10)
best_clf = clf.fit(X_train, y_train)
best_clf.best_estimator_
best_clf.best_score_

c:\Users\yvonn\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.6825264946201763

In [11]:
best_model = best_clf.best_estimator_
best_model.score(X_test,y_test)

0.6747720364741642

In [12]:
best_clf.best_params_
# {'C': 100, 'penalty': 'l2', 'solver': 'saga'}

{'C': 100, 'penalty': 'l2', 'solver': 'saga'}

In [13]:
import joblib

joblib.dump(best_clf.best_estimator_,'../models/best_model_logistic_regression.pkl')


['../models/best_model_logistic_regression.pkl']